In [36]:
# Load .env file
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path="../.env")

True

## Example Selector for Few-Shot Prompting
Anche se i modelli di grandi dimensioni dimostrano notevoli capacità in zero-shot, questi sono ancora carenti su compiti più complessi. 

Il prompting few-shot può essere utilizzato come tecnica per abilitare l'in-context learning in cui forniamo dimostrazioni nel prompt per indirizzare il modello verso una migliore prestazione.

Esistono differenti tipi di selector guarda [qui](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/) per una lista completa.



In [2]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [4]:
# SemanticSimilarityExampleSelector selezionerà esempi che sono semanticamente simili al nostro input
example_selector = SemanticSimilarityExampleSelector.from_examples(
	# Questa è la lista di esempi disponibili per la selezione.
    examples, 
    
	# Questa è la classe utilizzata per produrre embedding che vengono utilizzati per misurare la similarità semantica.
    OpenAIEmbeddings(), 

	# Questa è la classe VectorStore utilizzata per memorizzare gli embedding e fare una ricerca di similarità.
    FAISS, 
    
	# Questo è il numero di esempi da produrre.
    k=2
)

In [5]:
similar_prompt = FewShotPromptTemplate(
	# Questo è l'oggetto che aiuterà a selezionare gli esempi
    example_selector=example_selector,
    
    # Il tuo prompt
    example_prompt=example_prompt,
    
	# Customizzazioni che verranno aggiunte all'inizio e alla fine del tuo prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
	# Quale input riceverà il tuo prompt
    input_variables=["noun"],
)

In [8]:
# Scegli una parola!
my_word = "student"

print(similar_prompt.format(noun=my_word))

Give the location an item is usually found in

Example Input: driver
Example Output: car

Example Input: pilot
Example Output: plane

Input: student
Output:


In [9]:
llm(similar_prompt.format(noun=my_word))

' classroom'

## Output Parser

Molto spesso i modelli generano output che non sono direttamente utilizzabili. Per consentire ciò, questi output devono essere analizzati e convertiti in un formato utilizzabile, come un testo JSON o un oggetto Python.

In Langchain, questo processo è gestito da un output parser. Un output parser è un oggetto che prende in input un testo generato dal modello e restituisce un oggetto Python.

Due concetti principali:

**1. Format Instructions** - Un prompt auto-generato che istruisce il modello su come formattare l'output.

**2. Parser** - Un metodo che prende in input il testo generato dal modello e restituisce un oggetto Python (o json).

Ci sono diversi tipi output parser. Noi vedremo:
- **List parser**
- **Pydantic (JSON) parser**

### List Parser

In [16]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [17]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [20]:
formatted_prompt = prompt.format(subject="dogs breeds")
output = llm(formatted_prompt)

output_parser.parse(output)

['Poodle', 'Labrador Retriever', 'German Shepherd', 'Beagle', 'Bulldog']

### Pydantic (JSON) Parser

In [21]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [23]:
# Definiamo la classe della struttura desiderata
class Animal(BaseModel):
	name: str = Field(description="The name of the animal")
	species: str = Field(description="The species of the animal")
	favorite_foods: List[str] = Field(description="The favorite foods of the animal")

# Definiamo il parser
output_parser = PydanticOutputParser(pydantic_object=Animal)

# Definiamo il prompt
prompt = PromptTemplate(
	template="I want to know which animal lives in the {habitat}. \n\n {format_instructions}",
	input_variables=["habitat"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)


_input = prompt.format_prompt(habitat="forest")

output = llm(_input.to_string())

output_parser.parse(output)

Animal(name='Chipmunk', species='Tamias striatus', favorite_foods=['acorns', 'berries', 'seeds'])

### Che succede se il modello genera un output non valido?


In [24]:
# Wrong output with name missing and wrong format
wrong_output = """
{
  "species": "Panthera tigris"
  "favorite_foods": [
    "Deer",
    "Wild boar",
    "Water buffalo"
  ]	
} 
"""
output_parser.parse(wrong_output)

OutputParserException: Failed to parse Animal from completion 
{
  "species": "Panthera tigris"
  "favorite_foods": [
    "Deer",
    "Wild boar",
    "Water buffalo"
  ]	
} 
. Got: Expecting ',' delimiter: line 3 column 3 (char 35)

### Soluzione: Retry e Fixing Parser
Nel caso in cui il primo  output parser fallisca, chiama un altro LLM per correggere eventuali errori.

- **Fixing** - Riprova a generare per fixare l'output con un altro parser.
- **Retry** - Riprova a generare l'output con lo stesso prompt.

In [25]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

new_parser.parse(wrong_output)

Animal(name='Tiger', species='Panthera tigris', favorite_foods=['Deer', 'Wild boar', 'Water buffalo'])

In [29]:
from langchain.output_parsers import RetryWithErrorOutputParser

wrong_output = """
{
  "species": "Panthera tigris"
  "favorite_foods": [
  ]	
} 
"""

retry_parser = RetryWithErrorOutputParser.from_llm(parser=output_parser, llm=llm)
retry_parser.parse_with_prompt(wrong_output, _input )

Animal(name='Tiger', species='Panthera tigris', favorite_foods=['Deer', 'Pig', 'Fish'])

## Memory
Aiutare gli LLM a ricordare le informazioni.


Memory è un termine un po' vago. Potrebbe essere utilizzata per ricordare le informazioni di cui hai parlato in passato o il recupero di informazioni più complesse.

Noi ci concentreremo sul primo caso, ovvero ricordare le informazioni di cui hai parlato in passato.

Ci sono molti tipi di memory, esplora [la documentazione](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html) per vedere quale si adatta al tuo caso d'uso.

### Chat Message History

In [31]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

In [32]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False)]

In [33]:
ai_response = chat(history.messages)
ai_response

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)

In [34]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)]

## Indexes - Struttura i documenti per la memorizzazione di informazioni

### **Document Loaders**

Modo facile per caricare i documenti da varie sorgenti. Langchain offre una [lista estesa](https://python.langchain.com/docs/modules/data_connection/document_loaders.html) di documents loader, ancora di più possono essere trovati su [Llama Index](https://llama-hub-ui.vercel.app/).

In [62]:
# Proviamo a caricare un file pdf

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("../documents/Jailbreaking_ChatGPT_paper.pdf")
pages = loader.load_and_split()
len(pages)

23

In [64]:
pages[0]

Document(page_content='Jailbreaking ChatGPT via Prompt Engineering: An\nEmpirical Study\nYi Liu∗, Gelei Deng∗, Zhengzi Xu∗, Yuekang Li†, Yaowen Zheng∗, Ying Zhang‡, Lida Zhao∗,\nTianwei Zhang∗, Yang Liu∗\n∗Nanyang Technological University , Singapore\n†University of New South Wales , Australia\n‡Virginia Tech , USA\nAbstract —Large Language Models (LLMs), like C HATGPT,\nhave demonstrated vast potential but also introduce challenges\nrelated to content constraints and potential misuse. Our study\ninvestigates three key research questions: (1) the number of dif-\nferent prompt types that can jailbreak LLMs, (2) the effectiveness\nof jailbreak prompts in circumventing LLM constraints, and (3)\nthe resilience of C HATGPT against these jailbreak prompts.\nInitially, we develop a classification model to analyze the distri-\nbution of existing prompts, identifying ten distinct patterns and\nthree categories of jailbreak prompts. Subsequently, we assess\nthe jailbreak capability of prompts wi

In [61]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How to jailbreak chatgpt?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:500])

0: Jailbreaking ChatGPT via Prompt Engineering: An
Empirical Study
Yi Liu∗, Gelei Deng∗, Zhengzi Xu∗, Yuekang Li†, Yaowen Zheng∗, Ying Zhang‡, Lida Zhao∗,
Tianwei Zhang∗, Yang Liu∗
∗Nanyang Technological University , Singapore
†University of New South Wales , Australia
‡Virginia Tech , USA
Abstract —Large Language Models (LLMs), like C HATGPT,
have demonstrated vast potential but also introduce challenges
related to content constraints and potential misuse. Our study
investigates three key research
0: tions. For instance, a common way to jailbreak C HATGPT
through prompts is to instruct it to emulate a "Do Anything
Now" (DAN) behavior [9]. This approach allows C HATGPT
to produce results that were previously unattainable.
In response to prompt engineering-based jailbreaking at-
tempts, OpenAI has imposed more strict rules [10] to pro-
hibit the use of such prompts. However, due to the inherent
flexibility of natural languages, there are multiple ways to
construct prompts that convey th

### **Retrievers**
Un modo semplice per combinare i documenti con i modelli di linguaggio.


Ci sono molti tipi di retrievers, il più supportato è il VectoreStoreRetriever.

In [65]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [66]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(pages)

# Get embedding engine ready
embeddings = OpenAIEmbeddings()

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [67]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [68]:
retriever

VectorStoreRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7fc00290ef10>, search_type='similarity', search_kwargs={})

In [70]:
docs = retriever.get_relevant_documents("what types of different techniques did the author mentioned?")
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

refer to the jailbreak question throughout the paper.
2

Ⅰ. Pretending
(97.44%, 76)
Ⅲ. Attention Shifting
(6.41%, 5)
Ⅱ. Privilege Escalation
(17.95%, 14)A. Character  Role Play
(89.74%, 70)B. Assumed
Responsibility
(79.49%, 62)C. Resear ch Experiment  
(2.5


In [71]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [72]:
embedding_list = embeddings.embed_documents([text.page_content for text in pages])

## Agents 🤖
La documentazione ufficiale di LangChain descrive perfettamente gli agenti:
> Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an **unknown chain** that depends on the user's input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then **decide which, if any, of these tools to call**.

In pratica si utilizza un LLM non solo per l'output di testo, ma anche per la presa di decisioni. 
La potenza di questa funzionalità è enorme. 

Sam Altman, CEO di OpenAI, ha affermato che i modelli di grandi dimensioni sono ottimi '[reasoning engine](https://www.youtube.com/watch?v=L_Guz73e6fw&t=867s)'. Gli Agent traono vantaggio da questo.

**Main Concepts**:

- **Agent** - Un oggetto che prende in input un testo e restituisce un oggetto Python.
- **Tools** - Le 'capacità' di un agente. Astrazione costruita sopra una funzione che ne permette facilmente la chiamata per una LLM (e un agente). Ex: Google Search, Calculator, etc.
- **Toolkits** - Gtuppi di tool che possono essere utilizzati da un agente.

In [53]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
import os, json

Create an API key for the SERO tool (https://serpapi.com/)

In [41]:
serpapi_api_key = os.getenv("SERP_API_KEY")

In [43]:
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key)

Esistono vari tipi di Agenti, noi vedremo solo l'agente che utilizza il [ReAct](https://react-lm.github.io/) framework.

ReAct si ispire ai concetti di "ragionare" (Reasoning) e "agire" (Act) che permettono agli esseri umani di imparare nuovi compiti e prendere decisioni o ragionare.

![ReAct](../documents/react.webp)

In [44]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [51]:
response = agent({"input":"What is the technical specification and the price of the new Apple Vision announced by Apple?"})



> Entering new  chain...
 Doing research on this topic will help me answer this question 
Action: Search 
Action Input: "Apple Vision technical specification"
Observation: The device contains a 1440×1600 pixel display, 90Hz refresh, Qualcomm Snapdragon 845 chip, 4GB of RAM, 64GB of storage, eye/hand tracking, passthrough technology, spatial audio, and no built-in battery.
Thought: I should look at the price of the device
Action: Search 
Action Input: "Apple Vision price"
Observation: $3,499
Thought: I now have the technical specification and price of the Apple Vision
Final Answer: The technical specification of the Apple Vision includes a 1440×1600 pixel display, 90Hz refresh, Qualcomm Snapdragon 845 chip, 4GB of RAM, 64GB of storage, eye/hand tracking, passthrough technology, spatial audio, and no built-in battery. The price is $3,499.

> Finished chain.


In [56]:
from pprint import pprint
pprint(response["intermediate_steps"])

[(AgentAction(tool='Search', tool_input='Apple Vision technical specification', log=' Doing research on this topic will help me answer this question \nAction: Search \nAction Input: "Apple Vision technical specification"'),
  'The device contains a 1440×1600 pixel display, 90Hz refresh, Qualcomm '
  'Snapdragon 845 chip, 4GB of RAM, 64GB of storage, eye/hand tracking, '
  'passthrough technology, spatial audio, and no built-in battery.'),
 (AgentAction(tool='Search', tool_input='Apple Vision price', log=' I should look at the price of the device\nAction: Search \nAction Input: "Apple Vision price"'),
  '$3,499')]
